Voglio modellare un solo pullman con 10 posti e clienti che arrivano ogni 1 tu.

1) Il bus è in attesa e appena salgono 10 cienti parte.
2) impiega 12 tu ad andare/scaricare e tornare.
3) Carica altri 10 clienti e riparte.

In [1]:
import simpy

In [ ]:
class Bus(object):
    def __init__(self, env, bus_capacity):
        self.env = env
        self.capacity = simpy.Container(env, capacity=bus_capacity, init=0)
        self.action = env.process(self.run())

    def run(self):
        while True:
            # 1. ATTESA: Attendi che 10 passeggeri salgano.
            # L'operazione `get` si bloccherà finché il container non conterrà
            # abbastanza elementi da prelevare. In questo caso, attende che
            # 10 passeggeri abbiano fatto `put`.
            print(f'{self.env.now:.2f}: Il bus è alla fermata. Posti liberi: {self.capacity.capacity - self.capacity.level}')
            yield self.capacity.get(self.capacity.capacity) # sto tentando di prendere tutti i posti del bus, quindi questa non si sbloccherà finché il bus non sarà pieno.
            
            # Se il codice arriva qui, significa che 10 passeggeri sono saliti.
            
            # 2. PARTENZA E VIAGGIO
            print(f'{self.env.now:.2f}: Bus pieno, partenza!')
            yield self.env.timeout(12)  # Durata del viaggio
            
            # 3. RITORNO E SCARICO
            print(f'{self.env.now:.2f}: Bus tornato alla fermata. Svuoto i posti.')
            # Rimettiamo i "posti" a disposizione nel container.
            yield self.capacity.put(self.capacity.capacity)

In [81]:
env = simpy.Environment()
# Create a bus with a capacity of 10 passengers
bus = Bus(env,10)

Starting passengers:  0
Bus capacity:  10


In [82]:
# Funzione che definisce il comportamento di un passeggero
def passeggero(env, nome, bus):
    """
    Il processo di un passeggero: arriva e cerca di salire sul bus.
    """
    print(f'{env.now:.2f}: Il passeggero {nome} arriva alla fermata.')
    
    # Il passeggero tenta di "mettere" se stesso (1) nel container dei posti.
    # Questa è l'azione chiave!
    yield bus.capacity.put(1)
    
    # Se il codice arriva qui, significa che il passeggero è riuscito a salire.
    print(f'{env.now:.2f}: Il passeggero {nome} è salito. Posti occupati: {bus.capacity.level}')

In [83]:
def arrivi_passeggeri(env, bus):
    """
    Genera un nuovo passeggero ogni 1 minuto.
    """
    contatore_passeggeri = 0
    while True:
        # Attendi 1 minuto prima di creare il prossimo passeggero
        yield env.timeout(1)
        
        contatore_passeggeri += 1
        # Crea un nuovo processo per il nuovo passeggero
        env.process(passeggero(env, f'P{contatore_passeggeri}', bus))

In [62]:
print(bus.capacity._capacity)
passenger_arrival_process = env.process(passenger_arrival(env, bus))
env.run(until=50)  # Run the simulation for 50 time units
print(bus.capacity._capacity)


10
Bus waiting for passengers at 0 with 0 passengers
Passenger boarded at 0
Bus waiting for passengers at 1 with 0 passengers
Passenger boarded at 1
Bus waiting for passengers at 2 with 0 passengers
Passenger boarded at 2
Bus waiting for passengers at 3 with 0 passengers
Passenger boarded at 3
Bus waiting for passengers at 4 with 0 passengers
Passenger boarded at 4
Bus waiting for passengers at 5 with 0 passengers
Passenger boarded at 5
Bus waiting for passengers at 6 with 0 passengers
Passenger boarded at 6
Bus waiting for passengers at 7 with 0 passengers
Passenger boarded at 7
Bus waiting for passengers at 8 with 0 passengers
Passenger boarded at 8
Bus waiting for passengers at 9 with 0 passengers
Passenger boarded at 9
Bus waiting for passengers at 10 with 0 passengers
Passenger boarded at 10
Bus waiting for passengers at 11 with 0 passengers
Passenger boarded at 11
Bus waiting for passengers at 12 with 0 passengers
Passenger boarded at 12
Bus waiting for passengers at 13 with 0 pa